In [1]:
from PIL import Image, ImageDraw, ImageFont
import random
import glob
import os

def augment_item(item):
    angle = random.randint(0, 1)
    scale = random.uniform(0.6, 1.4)
    rotated_item = item.rotate(angle, expand=True, fillcolor=(255, 255, 255))
    scaled_width = int(rotated_item.width * scale)
    scaled_height = int(rotated_item.height * scale)
    scaled_item = rotated_item.resize((scaled_width, scaled_height), Image.ANTIALIAS)
    return scaled_item

def add_noise(draw, canvas_size):
    # Add random lines
    num_lines = random.randint(5, 10)
    for _ in range(num_lines):
        start = (random.randint(0, canvas_size[0]), random.randint(0, canvas_size[1]))
        end = (random.randint(0, canvas_size[0]), random.randint(0, canvas_size[1]))
        draw.line([start, end], fill=(0, 0, 0), width=3)
    
    # Add random text
    num_texts = random.randint(5, 10)
    try:
        font = ImageFont.truetype("arial.ttf", 22)
    except IOError:
        font = ImageFont.load_default()
    for _ in range(num_texts):
        text = "".join(random.choices('abcdefghijkl mnopqrstuvwxyzABCDEF GHIJKLMNOP QRSTUVWXYZ0123456789', k=random.randint(5, 15)))
        position = (random.randint(0, canvas_size[0] - 100), random.randint(0, canvas_size[1] - 30))
        draw.text(position, text, font=font, fill=(0, 0, 0))

def get_edge_point(x, y, w, h):
    edge_choice = random.choice(['top', 'bottom', 'left', 'right'])
    if edge_choice == 'top':
        return (x + random.randint(0, w), y)
    elif edge_choice == 'bottom':
        return (x + random.randint(0, w), y + h)
    elif edge_choice == 'left':
        return (x, y + random.randint(0, h))
    else:  # 'right'
        return (x + w, y + random.randint(0, h))

def add_arrows(draw, placed_items):
    num_arrows = random.randint(3, 20)
    for _ in range(num_arrows):
        if len(placed_items) < 2:
            break
        item1, item2 = random.sample(placed_items, 2)
        x1, y1, w1, h1 = item1
        x2, y2, w2, h2 = item2
        start = get_edge_point(x1, y1, w1, h1)
        end = get_edge_point(x2, y2, w2, h2)
        
        if end[0] > start[0]:  # Rightward arrow
            elbow_x = end[0] - random.randint(10, 20)
            elbow_y1 = start[1]
            elbow_y2 = end[1]
        elif end[0] < start[0]:  # Leftward arrow
            elbow_x = end[0] + random.randint(10, 20)
            elbow_y1 = start[1]
            elbow_y2 = end[1]
        elif end[1] > start[1]:  # Downward arrow
            elbow_y1 = end[1] - random.randint(10, 20)
            elbow_x = start[0]
            elbow_x2 = end[0]
        else:  # Upward arrow
            elbow_y1 = end[1] + random.randint(10, 20)
            elbow_x = start[0]
            elbow_x2 = end[0]
        
        arrow_thickness = random.randint(1, 5)
        if end[0] != start[0]:  # Horizontal arrow
            draw.line([start, (elbow_x, elbow_y1)], fill=(0, 0, 0), width=arrow_thickness)
            draw.line([(elbow_x, elbow_y1), (elbow_x, elbow_y2)], fill=(0, 0, 0), width=arrow_thickness)
            draw.line([(elbow_x, elbow_y2), end], fill=(0, 0, 0), width=arrow_thickness)
        else:  # Vertical arrow
            draw.line([start, (elbow_x, elbow_y1)], fill=(0, 0, 0), width=arrow_thickness)
            draw.line([(elbow_x, elbow_y1), (elbow_x2, elbow_y1)], fill=(0, 0, 0), width=arrow_thickness)
            draw.line([(elbow_x2, elbow_y1), end], fill=(0, 0, 0), width=arrow_thickness)

        # Add arrowhead
        arrow_head_length = 10
        if end[0] > start[0]:  # Rightward arrow
            draw.polygon([(end[0], end[1]),
                          (end[0] - arrow_head_length, end[1] - arrow_head_length // 2),
                          (end[0] - arrow_head_length, end[1] + arrow_head_length // 2)], fill=(0, 0, 0))
        elif end[0] < start[0]:  # Leftward arrow
            draw.polygon([(end[0], end[1]),
                          (end[0] + arrow_head_length, end[1] - arrow_head_length // 2),
                          (end[0] + arrow_head_length, end[1] + arrow_head_length // 2)], fill=(0, 0, 0))
        elif end[1] > start[1]:  # Downward arrow
            draw.polygon([(end[0], end[1]),
                          (end[0] - arrow_head_length // 2, end[1] - arrow_head_length),
                          (end[0] + arrow_head_length // 2, end[1] - arrow_head_length)], fill=(0, 0, 0))
        else:  # Upward arrow
            draw.polygon([(end[0], end[1]),
                          (end[0] - arrow_head_length // 2, end[1] + arrow_head_length),
                          (end[0] + arrow_head_length // 2, end[1] + arrow_head_length)], fill=(0, 0, 0))

def generate_synthetic_images(image_dir, output_dir, num_images=10, canvas_size=(1024, 1024)):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    item_paths = glob.glob(os.path.join(image_dir, "*.png"))
    items = [Image.open(item) for item in item_paths]
    
    for i in range(num_images):
        random.shuffle(items)
        canvas = Image.new('RGB', canvas_size, (255, 255, 255))
        draw = ImageDraw.Draw(canvas)
        label_data = []
        placed_items = []

        for item in items:
            augmented_item = augment_item(item)
            item_width, item_height = augmented_item.size
            max_x = max(0, canvas_size[0] - item_width)  # Ensure non-negative
            max_y = max(0, canvas_size[1] - item_height)  # Ensure non-negative

            if max_x == 0 or max_y == 0:
                continue  # Skip if the augmented item is too large

            placed = False
            attempts = 0
            while not placed and attempts < 100:
                if max_x > 0 and max_y > 0:
                    x = random.randint(0, max_x)
                    y = random.randint(0, max_y)
                else:
                    break  # No valid placement if dimensions don't fit

                overlap = False
                for placed_item in placed_items:
                    px, py, pw, ph = placed_item
                    if not (x + item_width < px or x > px + pw or y + item_height < py or y > py + ph):
                        overlap = True
                        break

                if not overlap:
                    placed_items.append((x, y, item_width, item_height))
                    placed = True

                attempts += 1

            if not placed:
                continue  # Skip placing this item

            canvas.paste(augmented_item, (x, y), augmented_item if augmented_item.mode == 'RGBA' else None)
            x_center = (x + item_width / 2) / canvas_size[0]
            y_center = (y + item_height / 2) / canvas_size[1]
            width = item_width / canvas_size[0]
            height = item_height / canvas_size[1]
            label_data.append(f"0 {x_center} {y_center} {width} {height}")

        # Adding arrows connecting items
        add_arrows(draw, placed_items)

        # Adding noise after all items have been placed
        add_noise(draw, canvas_size)

        image_save_path = os.path.join(output_dir, f"synthetic_image_{i}.jpg")
        canvas.save(image_save_path)
        label_save_path = os.path.join(output_dir, f"synthetic_image_{i}.txt")
        with open(label_save_path, 'w') as label_file:
            label_file.write("\n".join(label_data))

# Usage example
generate_synthetic_images('../components', 'output', num_images=400, canvas_size=(1024, 1024))


C:\Users\NajibS\AppData\Local\Temp\ipykernel_23784\4203935059.py:12: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  scaled_item = rotated_item.resize((scaled_width, scaled_height), Image.ANTIALIAS)
